In [1]:
import pandas as pd
import glob

#modify the file path below - give wherever your files are present
path = r'LDA' # use your path
all_files = glob.glob(path + "/*.txt")

li = []


In [2]:
all_files

['C:\\Users\\nibed\\Desktop\\Freelance\\LDA\\aa_0001675149_EX99_20170124_EX-99_excerpt.txt',
 'C:\\Users\\nibed\\Desktop\\Freelance\\LDA\\a_0001090872_EX991_20141117_EX-99_excerpt.txt',
 'C:\\Users\\nibed\\Desktop\\Freelance\\LDA\\a_0001090872_EX991_20170214_EX-99_excerpt.txt']

In [48]:
#read text file

for filename in all_files:
    
    import csv
    d1 = pd.read_csv(filename,delimiter = "\t",header=None,error_bad_lines=False,sep='delimiter', quoting=csv.QUOTE_NONE)
    d1.columns = ['Text']
    
    import nltk
    import nltk
    import nltk
    from nltk.tokenize import RegexpTokenizer
    from nltk.stem import WordNetLemmatizer,PorterStemmer
    from sklearn.feature_extraction.text import TfidfVectorizer

    from nltk.corpus import stopwords
    import re
    import numpy as np
    import pandas as pd
    import re, nltk, gensim
    # Sklearn
    from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    from sklearn.model_selection import GridSearchCV
    from pprint import pprint

    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer() 

    nltk.download('wordnet')
    
    #break to sentences
    #p is the list of sentences
    p = d1['Text'].values.tolist()
    k=''.join(p)
    p=[]
    sent_text = nltk.sent_tokenize(k)
    for it in sent_text:
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(it)  
        filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
        mon =["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        filtered_words = [w for w in filtered_words if w not in mon]
        word = list(set(filtered_words))
        lemma_words=[lemmatizer.lemmatize(w) for w in word]
        p.append(' '.join(lemma_words))
        
    def getTopicPerSentence(p):
        if len(p)>50:
            vectorizer = TfidfVectorizer(analyzer='word',       
                                        # min_df=5, max_df = 1.0,
            # minimum reqd occurences of a word 
                                         stop_words='english',             
            # remove stop words
                                         lowercase=True,                   
            # convert all words to lowercase
                                         token_pattern='[a-zA-Z]{3,}')
            data_vectorized = vectorizer.fit_transform(p)
        else:
            vectorizer = TfidfVectorizer(analyzer='word',       
                                        # min_df=5, max_df = 1.0,
            # minimum reqd occurences of a word 
                                         stop_words='english',             
            # remove stop words
                                         lowercase=True,                   
            # convert all words to lowercase
                                         token_pattern='[a-zA-Z]{3,}')
            data_vectorized = vectorizer.fit_transform(p)
            
        
        lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               
        # Max learning iterations
                                              learning_method='online',   
                                              random_state=100,          
        # Random state
                                              batch_size=128,            
        # n docs in each learning iter
                                              evaluate_every = -1,       
        # compute perplexity every n iters, default: Don't
                                              n_jobs = -1,               
        # Use all available CPUs
                                             )
        lda_output = lda_model.fit_transform(data_vectorized)
        
        # Create Document — Topic Matrix
        lda_output =lda_model.transform(data_vectorized)
        # column names
        topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
        # index names
        docnames = ["Doc" + str(i) for i in range(len(p))]
        # Make the pandas dataframe
        df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
        # Get dominant topic for each document
        dominant_topic = np.argmax(df_document_topic.values, axis=1)
        df_document_topic["dominant_topic"] = dominant_topic
        # Styling
        def color_green(val):
         color = 'green' if val > .1 else 'black'
         return 'color: {col}'.format(col=color)
        def make_bold(val):
         weight = 700 if val > .1 else 400
         return 'font-weight: {weight}'.format(weight=weight)
        # Apply Style
        df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
        
        dd= pd.DataFrame(dominant_topic)
        dd.columns =['Topic']
        k = [i for i in range(1,len(p)+1)]
        dd['Document']=k
        
        # Topic-Keyword Matrix
        df_topic_keywords = pd.DataFrame(lda_model.components_)
        # Assign Column and Index
        df_topic_keywords.columns = vectorizer.get_feature_names()
        df_topic_keywords.index = topicnames
        
        # Show top n keywords for each topic
        def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
            keywords = np.array(vectorizer.get_feature_names())
            topic_keywords = []
            for topic_weights in lda_model.components_:
                top_keyword_locs = (-topic_weights).argsort()[:n_words]
                topic_keywords.append(keywords.take(top_keyword_locs))
            return topic_keywords
        topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5)
        # Topic - Keywords Dataframe
        df_topic_keywords = pd.DataFrame(topic_keywords)
        df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
        df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
        df_topic_keywords['List']=df_topic_keywords.values.tolist()
        
        df_topic_keywords = df_topic_keywords[['List']]
        p1 =[i for i in range(0,20)]
        df_topic_keywords['Topic']=p1
        
        dm = pd.merge(dd,df_topic_keywords,on='Topic',how='left')
        dm.drop(['Topic'],axis=1,inplace=True)
        dm.columns =['Sentence Number','Topic']
        
        return dm
    #get the dataframe
    fil = getTopicPerSentence(p)
    
    #save it as csv
    n = str(str(filename).split('\\')[-1]).split('.')[0]
    
    #save file to the current working directory
    fil.to_csv("{}.csv".format(n),index=False)
    


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

...

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Us

In [102]:
getTopicPerSentence(p)

,Sentence Number,Topic
0,1,"[company, financial, corporation, alcoa, carve]"
1,2,"[prepared, basis, fourth, carve, financial]"
2,3,"[company, financial, corporation, alcoa, carve]"
3,4,"[information, additional, united, states, secu..."
4,5,"[alcoa, aluminum, statement, item, special]"
...,...,...
122,123,"[company, financial, corporation, alcoa, carve]"
123,124,"[company, financial, corporation, alcoa, carve]"
124,125,"[company, financial, corporation, alcoa, carve]"
125,126,"[information, additional, united, states, secu..."
